In [1]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
embedding_dim = 728

In [3]:
tokenized_lines = open("input.txt", "r")
tokenized_lines = tokenized_lines.readlines()

vocab = set()
special_tokens = ["<pad>", "<start>", "<end>"]
for sentence in tokenized_lines:
    vocab.update(sentence.split())
vocab = special_tokens + list(vocab)

vocab_to_index = {word:index for index, word in enumerate(vocab)}
vocab_size = len(vocab)
#print(vocab)
#print("Vocab size: ", vocab_size)

In [4]:
from torch.nn.utils.rnn import pad_sequence

PAD_TOKEN = "<pad>"
PAD_IDX = vocab_to_index[PAD_TOKEN]

def collate_batch(batch):
    inputs, targets = zip(*batch)

    #inputs = [torch.tensor(seq, dtype = torch.long()) for seq in inputs]
    #targets = [torch.tensor(seq, dtype = torch.long()) for seq in targets]

    padded_inputs = pad_sequence(inputs, batch_first=True, padding_value=PAD_IDX)
    padded_targets = pad_sequence(targets, batch_first=True, padding_value=PAD_IDX)

    return padded_inputs, padded_targets


# 1. Rebuild vocab from lowercased text and include <unk>
special_tokens = ["<pad>", "<start>", "<end>", "<unk>"]

vocab_to_index = {}

vocab = set()
for sentence in tokenized_lines:
    vocab.update(sentence.lower().split())      # lowercase here

vocab = special_tokens + sorted(vocab)          # sorted for reproducibility
vocab_to_index = {w:i for i,w in enumerate(vocab)}

PAD_IDX = vocab_to_index["<pad>"]
UNK_IDX = vocab_to_index["<unk>"]

# 2. Update your Dataset to use .get(…, UNK_IDX) instead of direct indexing
class ShakespeareDataset(Dataset):
    def __init__(self, tokenized_lines, vocab_to_idx):
        self.data = [
            line.lower().split()
            for line in tokenized_lines
            if len(line.lower().split()) > 2  # ignore short lines
        ]
        self.vocab_to_idx = vocab_to_idx

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        words = self.data[idx]

        # THIS should raise error if token is missing
        input_ids = [self.vocab_to_idx.get(word, self.vocab_to_idx["<unk>"]) for word in words[:-1]]
        target_ids = [self.vocab_to_idx.get(word, self.vocab_to_idx["<unk>"]) for word in words[1:]]

        return torch.tensor(input_ids, dtype=torch.long), torch.tensor(target_ids, dtype=torch.long)

In [5]:
def positional_encodings(seq_len, embedding_dim, device):
    position = torch.arange(seq_len, dtype=torch.float, device=device).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, embedding_dim, 2, device=device).float() * (-math.log(10000.0) / embedding_dim))
    pe = torch.zeros(seq_len, embedding_dim, device=device)
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    return pe

In [6]:
class self_attention(nn.Module):
    def __init__(self):
        super(self_attention, self).__init__()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, Q, K, V, attn_mask=None):
        # Q, K, V shape: (batch, seq_len, dim)
        batch_size, seq_len, dim = Q.size()

        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(dim)  # (batch, seq_len, seq_len)

        # Causal mask (upper triangular)
        causal_mask = torch.triu(torch.ones(seq_len, seq_len, device=Q.device) * float('-inf'), diagonal=1)
        scores = scores + causal_mask

        # Padding mask (optional)
        if attn_mask is not None:
            # attn_mask: (batch, 1, seq_len), 1 for keep, 0 for mask
            scores = scores.masked_fill(attn_mask == 0, float('-inf'))

        weights = self.softmax(scores)
        context = torch.matmul(weights, V)  # (batch, seq_len, dim)

        return context


In [7]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        # creating the multi-headed attention block.
        self.self_attn1 = self_attention()
        self.self_attn2 = self_attention()
        self.self_attn3 = self_attention()
        self.self_attn4 = self_attention()

        self.self_attn5 = self_attention()
        self.self_attn6 = self_attention()
        self.self_attn7 = self_attention()
        self.self_attn8 = self_attention()


        # All the layers, we gonna need to make the decoder work.
        self.layer_norm = nn.LayerNorm(embedding_dim)
        self.softmax = nn.Softmax(-1)
        
        #self.latent_downscale = nn.Linear(16, 2)

        self.w_q = nn.Linear(in_features=embedding_dim, out_features=91)
        #self.w_k = nn.Linear(in_features=16, out_features=2)
        #self.w_v = nn.Linear(in_features=16, out_features=2)
        #self.w_x = nn.Linear(in_features=16, out_features=2)

        self.latent_upscale = nn.Linear(91, embedding_dim)

        self.final_linear_layer = nn.Linear(embedding_dim, vocab_size)


    def forward(self, Q, K, V, X, attn_mask=None):
        q = self.w_q(Q)
        k = self.w_q(K)
        v = self.w_q(V)
        x = self.w_q(X)

        context1 = self.self_attn1(q, k, v, attn_mask)
        context2 = self.self_attn2(q, k, v, attn_mask)
        context3 = self.self_attn3(q, k, v, attn_mask)
        context4 = self.self_attn4(q, k, v, attn_mask)
        context5 = self.self_attn5(q, k, v, attn_mask)
        context6 = self.self_attn6(q, k, v, attn_mask)
        context7 = self.self_attn7(q, k, v, attn_mask)
        context8 = self.self_attn8(q, k, v, attn_mask)

        combined = torch.cat((context1, context2, context3, context4, context5, context6, context7, context8), 2)
        final_encodings = combined + self.latent_upscale(x)
        final_encodings = self.layer_norm(final_encodings)
        #logits = self.final_linear_layer(final_encodings)

        return final_encodings


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = Model().to(device)

In [9]:
embedding_layer = nn.Embedding(vocab_size, embedding_dim).to(device)
#model = Model().to(device)
PAD_IDX = vocab_to_index.get("<pad>", 0)  # Ensure this is consistent with your vocab
loss_fn = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = optim.Adam(model.parameters(), lr=1e-2)

# Assuming: tokenized_lines = open("input.txt").readlines(), vocab_to_idx built
dataset = ShakespeareDataset(tokenized_lines, vocab_to_index)
loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_batch)

def create_padding_mask(input_ids, pad_idx):
    input_ids: (batch, seq_len)
    return (input_ids != pad_idx).unsqueeze(1)  # (batch, 1, seq_len)

for epoch in range(100):
    total_loss = 0
    total_accuracy = 0

    for inputs, targets in loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Get embeddings
        input_embeddings = embedding_layer(inputs)   
        pos_enc = positional_encodings(input_embeddings.size(1), embedding_dim, device)
        input_with_pos = input_embeddings + pos_enc

        store_res = input_with_pos.shape[1]
        #print(store_res)

        input_with_pos = nn.Linear(input_with_pos.shape[1], 5).to(device)(input_with_pos.transpose(-2, -1))
        input_with_pos = input_with_pos.transpose(-2, -1)
        #print(input_with_pos.shape)
        
        final_encodings = model(input_with_pos, input_with_pos, input_with_pos, input_with_pos)
        #print(final_encodings.shape)
        final_encodings = nn.Linear(final_encodings.shape[1], store_res).to(device)(final_encodings.transpose(-2, -1))
        #print(final_encodings.shape)
        logits = nn.Linear(embedding_dim, vocab_size).to(device)(final_encodings.transpose(-2, -1))
        
        #print(final_encodings.shape)

        logits = logits.view(-1, vocab_size)
        targets = targets.view(-1)

        #print(logits.shape)
        #print(targets.shape)

        #loss = loss_fn(logits.view(-1, vocab_size), targets.view(-1))
        loss = loss_fn(logits, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accuracy
        predicted = torch.argmax(logits, dim=-1)
        correct = (predicted == targets).float()
        mask = (targets != PAD_IDX).float()
        accuracy = (correct * mask).sum() / mask.sum()

        total_loss += loss.item()
        total_accuracy += accuracy.item()

    avg_loss = total_loss / len(loader)
    avg_accuracy = total_accuracy / len(loader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}, Accuracy: {avg_accuracy:.4f}")
    #print(f"Epoch {epoch+1}, Loss: {avg_loss}, Accuracy: {avg_accuracy}")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (352x728 and 16x25673)

In [ ]:
total_accuracy

0.0

In [ ]:
for x, y in loader:
    print("Input:", x[0])
    print("Target:", y[0])
    break

Input: tensor([ 8026,  7466, 10343, 15940, 23576, 10378, 10343, 12107,     0])
Target: tensor([ 7466, 10343, 15940, 23576, 10378, 10343, 12107, 14649,     0])


In [ ]:
x.dtype

torch.int64

In [ ]:
logits

tensor([[[-0.3021, -0.7764,  0.5359,  ...,  0.4725, -0.1413, -0.4906],
         [-0.2999, -0.7784,  0.5250,  ...,  0.4617, -0.1388, -0.4856],
         [-0.1239, -0.8907,  0.3899,  ...,  0.2340, -0.0455, -0.6259],
         ...,
         [-0.7696,  0.6514,  0.5770,  ...,  0.4585, -0.3713,  0.2837],
         [-0.7342,  0.7084,  0.5475,  ...,  0.4854, -0.2788,  0.3135],
         [-0.6220,  0.6547,  0.7251,  ...,  0.6407, -0.2005,  0.0632]],

        [[ 0.0082,  0.3777,  0.9690,  ...,  0.7708,  0.2903, -0.8089],
         [-0.3768, -0.7505,  0.2232,  ...,  0.1718, -0.1880, -0.1934],
         [-0.4395, -0.7145,  0.1079,  ...,  0.0551, -0.2404, -0.0471],
         ...,
         [-0.5848, -0.6626,  0.1160,  ..., -0.0113, -0.4363,  0.0604],
         [-0.7534, -0.5014,  0.1725,  ...,  0.0251, -0.5941,  0.2111],
         [-0.8981, -0.0929,  0.4464,  ...,  0.2271, -0.7270,  0.2356]],

        [[ 0.0989, -0.8582,  0.7987,  ...,  0.5686,  0.1130, -1.1362],
         [-0.3044, -0.7957,  0.2549,  ...,  0

In [ ]:
import torch

def generate_sequence(model, start_text, vocab_to_idx, idx_to_vocab, embedding_layer, device, max_len=50):
    model.eval()  # Evaluation mode
    start_tokens = start_text.lower().split()

    # Convert words to indices
    input_ids = [vocab_to_idx.get(word, vocab_to_idx["<pad>"]) for word in start_tokens]
    generated = torch.tensor(input_ids, dtype=torch.long, device=device).unsqueeze(0)  # (1, seq_len)

    for _ in range(max_len):
        seq_len = generated.size(1)

        # Recalculate positional encodings each time
        pos = positional_encodings(seq_len, embedding_layer.embedding_dim, device)
        input_embed = embedding_layer(generated) + pos

        # Attention mask
        attn_mask = create_padding_mask(generated, vocab_to_idx["<pad>"]).to(device)

        with torch.no_grad():
            q = k = v = input_embed
            logits = model(q, k, v, input_embed, attn_mask)

        # Sample next token
        logits = logits[:, -1, :]  # Get last token's logits
        temperature = 0.7
        probs = torch.softmax(logits / temperature, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)  # Shape: (1, 1)

        # Stop if end token
        token_id = next_token.item()
        if idx_to_vocab.get(token_id, "") == "<end>":
            break

        # Append next token
        generated = torch.cat((generated, next_token), dim=1)

    # Convert generated indices back to words
    generated_text = ' '.join([idx_to_vocab.get(idx.item(), "<unk>") for idx in generated.squeeze()])
    return generated_text

# Example of inference usage:
start_text = "<start>"  # Starting text for generation
generated_text = generate_sequence(
    model=model, 
    start_text=start_text, 
    vocab_to_idx=vocab_to_index, 
    idx_to_vocab={index: word for word, index in vocab_to_index.items()}, 
    embedding_layer=embedding_layer, 
    device=device,
    max_len=50  # Limit generated sequence length
)

print("Generated Text:")
print(generated_text)


Generated Text:
<start> <pad> <start> <unk> &c. 'all <pad> 'almost '? ' &c: <start>
